In [1]:
import torch

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Custom CNN Network using VGG Blocks

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class CustomVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomVGG, self).__init__()
        
        # VGG blocks
        self.block1 = self._make_vgg_block(3, 64, 2)
        self.block2 = self._make_vgg_block(64, 128, 2)
        self.block3 = self._make_vgg_block(128, 256, 3)
        self.block4 = self._make_vgg_block(256, 512, 3)
        self.block5 = self._make_vgg_block(512, 512, 3)
        
        # Fully connected layers
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
        
        # Dropout layers
        self.dropout = nn.Dropout(0.5)
        
    def _make_vgg_block(self, in_channels, out_channels, num_convs):
        layers = []
        for _ in range(num_convs):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
            layers.append(nn.ReLU(inplace=True))
            in_channels = out_channels
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        
        x = torch.flatten(x, 1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        
        return x

## Data Loader

In [3]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = 'ImageNet/'
def get_data_loaders(data_dir, batch_size):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return train_loader, val_loader

## Optimizers

In [4]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import models

# # Define a function to load ResNet50 with modified output layer
# def load_resnet50(num_classes):
#     # Load pre-trained ResNet50
#     model =  models.resnet50(weights='ResNet50_Weights.DEFAULT')
#     # Modify the last fully connected layer to output 10 classes
#     num_features = model.fc.in_features
#     model.fc = nn.Linear(num_features, num_classes)
#     return model

# # Define a function to load VGG16 with modified output layer
# def load_vgg16(num_classes):
#     # Load pre-trained VGG16
#     model = models.vgg16(weights='VGG16_Weights.DEFAULT')
#     # Modify the last fully connected layer to output 10 classes
#     num_features = model.classifier[6].in_features
#     model.classifier[6] = nn.Linear(num_features, num_classes)
#     return model

In [5]:
import torch.optim as optim

# Function to create and return the model and optimizer
def create_model_optimizer(optimizer_name):
    model = CustomVGG(num_classes=10).to(device)
    # model = load_vgg16(num_classes=10).to(device)
    optimizers = {
        'SGD': optim.SGD(model.parameters(), lr=0.001, momentum=0.9),
        'Adam': optim.Adam(model.parameters(), lr=0.0001),
        'RMSprop': optim.RMSprop(model.parameters(), lr=0.0001),
        'AdamW': optim.AdamW(model.parameters(), lr=0.0001),
        'Adamax': optim.Adamax(model.parameters(), lr=0.0001),
        'SparseAdam': optim.SparseAdam(model.parameters(), lr=0.0001),
        'RAdam': optim.RAdam(model.parameters(), lr=0.0001),
        'NAdam': optim.NAdam(model.parameters(), lr=0.0001)
    }
    optimizer = optimizers[optimizer_name]
    return model, optimizer

# Training Loop

In [9]:
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

# Function to train and evaluate the model
def train_and_evaluate(optimizer_name, train_loader, val_loader):
    model, optimizer = create_model_optimizer(optimizer_name)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    scaler = GradScaler()

    best_val_acc = 0
    best_model_wts = None
    patience = 5
    trigger_times = 0

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    # Initialize lists to store epoch-wise data
    epoch_data = {
        'Epoch': [],
        'Train Loss': [],
        'Train Acc': [],
        'Val Loss': [],
        'Val Acc': []
    }

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        loop = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{num_epochs}]', leave=True)
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loop.set_postfix(loss=loss.item())
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        val_loss, val_correct, val_total = 0.0, 0, 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        scheduler.step()

        # Add epoch-wise data to the dictionary
        epoch_data['Epoch'].append(epoch + 1)
        epoch_data['Train Loss'].append(train_loss)
        epoch_data['Train Acc'].append(train_accuracy)
        epoch_data['Val Loss'].append(val_loss)
        epoch_data['Val Acc'].append(val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_model_wts = model.state_dict().copy()
            trigger_times = 0
        else:
            trigger_times += 1

        if trigger_times >= patience:
            print('Early stopping triggered')
            break

    print(f'Best Validation Accuracy: {best_val_acc:.2f}%')
    if best_model_wts:
        model.load_state_dict(best_model_wts)
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_acc': best_val_acc
    }, f'model_{optimizer_name}_batch_size_{batch_size}.pth')

    # Convert epoch-wise data dictionary to DataFrame
    df = pd.DataFrame(epoch_data)

    # Save DataFrame to CSV file
    df.to_csv(f'epoch_data_{optimizer_name}_batch_size_{batch_size}.csv', index=False)

    return train_losses, val_losses, train_accuracies, val_accuracies

In [10]:
# List of optimizers to be used
# optimizers_list = ['SGD', 'Adam', 'RMSprop', 'AdamW', 'Adamax', 'SparseAdam', 'RAdam', 'NAdam']
optimizers_list = ['SGD', 'Adam', 'RMSprop', 'AdamW', 'Adamax', 'RAdam', 'NAdam']
# optimizers_list = ['RAdam', 'NAdam']

# List of batch sizes to be used
batch_sizes = [8, 16, 32]

num_epochs = 10

# Train and evaluate for different optimizers and batch sizes
loss_dict, acc_dict = {}, {}

for optimizer_name in optimizers_list:
    for batch_size in batch_sizes:
        print(f'Training with optimizer {optimizer_name} and batch size {batch_size}')
        train_loader, val_loader = get_data_loaders(data_dir, batch_size)
        train_losses, val_losses, train_accuracies, val_accuracies = train_and_evaluate(optimizer_name, train_loader, val_loader)
        key = f'{optimizer_name}_bs{batch_size}'
        loss_dict[key] = {'train': train_losses, 'val': val_losses}
        acc_dict[key] = {'train': train_accuracies, 'val': val_accuracies}

Training with optimizer SGD and batch size 8


Epoch [1/10]: 100%|████████████████████████████████████████████████████████| 125/125 [00:13<00:00,  9.59it/s, loss=2.3]


Epoch [1/10] - Train Loss: 2.3026, Train Acc: 9.60%, Val Loss: 2.3026, Val Acc: 10.00%


Epoch [2/10]: 100%|████████████████████████████████████████████████████████| 125/125 [00:13<00:00,  9.55it/s, loss=2.3]


Epoch [2/10] - Train Loss: 2.3028, Train Acc: 9.30%, Val Loss: 2.3026, Val Acc: 10.00%


...

Epoch [4/10] - Train Loss: 2.3033, Train Acc: 9.50%, Val Loss: 2.3020, Val Acc: 10.00%


Epoch [5/10]: 100%|██████████████████████████████████████████████████████████| 32/32 [00:10<00:00,  3.14it/s, loss=2.3]


Epoch [5/10] - Train Loss: 2.3032, Train Acc: 9.00%, Val Loss: 2.3018, Val Acc: 10.00%


Epoch [6/10]: 100%|█████████████████████████████████████████████████████████| 32/32 [00:10<00:00,  3.15it/s, loss=2.31]


Epoch [6/10] - Train Loss: 2.3028, Train Acc: 9.70%, Val Loss: 2.3018, Val Acc: 10.00%
Early stopping triggered
Best Validation Accuracy: 10.00%
